In [1]:
import Pkg;

In [2]:
Pkg.add("ChainRules")

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`


In [3]:
using ChainRules
using IRTools

In [4]:
function mult(a, b)
    a * b
end

mult (generic function with 1 method)

In [5]:
function add(a, b)
    a + b
end

add (generic function with 1 method)

In [6]:
function composite(a, b)
    mult(a, add(a, b))
end

composite (generic function with 1 method)

In [7]:
bs = (IRTools.@code_ir composite(3, 5)).blocks[1]

IRTools.Inner.BasicBlock(IRTools.Inner.Statement[IRTools.Inner.Statement(:(Main.add(%2, %3)), Any, 1), IRTools.Inner.Statement(:(Main.mult(%2, %4)), Any, 1)], Any[%1, %2, %3], Any[Any, Any, Any], IRTools.Inner.Branch[return %5])

In [8]:
add_ref = (bs.stmts[1].expr.args[1])

:(Main.add)

In [9]:
add_ref

:(Main.add)

In [10]:
using ChainRules

In [11]:
ChainRules.rrule(add_ref, :x, :y)

In [12]:
ChainRules.rrule(typeof(Main.:*), :x, :y)

In [13]:
ChainRules.rrule(typeof(Main.sin), :x)

In [14]:
val, pullback = ChainRules.rrule(*, 3, 5)

(15, ChainRules.var"#times_pullback2#1333"{Int64, Int64}(3, 5))

In [15]:
pb = ChainRules.rrule(*, 3, 5, 6)[2](1)

(ChainRulesCore.NoTangent(), 30.0, 18.0, 15.0)

In [16]:
pullback(4)

(ChainRulesCore.NoTangent(), 20.0, 12.0)

In [17]:
+

+ (generic function with 243 methods)

In [18]:
eval(add_ref)

add (generic function with 1 method)

In [19]:
ChainRules.rrule(eval(add_ref), 3, 5)

In [20]:
val, pullback = ChainRules.rrule(add_ref, 3, 5)

LoadError: MethodError: no method matching iterate(::Nothing)
[0mClosest candidates are:
[0m  iterate([91m::Union{LinRange, StepRangeLen}[39m) at range.jl:872
[0m  iterate([91m::Union{LinRange, StepRangeLen}[39m, [91m::Integer[39m) at range.jl:872
[0m  iterate([91m::T[39m) where T<:Union{Base.KeySet{<:Any, <:Dict}, Base.ValueIterator{<:Dict}} at dict.jl:712
[0m  ...

In [21]:
pullback(5)

(ChainRulesCore.NoTangent(), 25.0, 15.0)

In [22]:
function simple_math(x, y)
    x * y + sin(x)
end

simple_math (generic function with 1 method)

In [23]:
gradient_2(simple_math, 3, 5)

LoadError: UndefVarError: gradient_2 not defined

In [39]:
@generated function gen_test(x)
    Core.println(x)
    return :(x * x)
end

gen_test (generic function with 1 method)

In [40]:
gen_test(5)

Int64


25

In [73]:
@generated function derivative_rule_gen(f, args, i)
    return :(gradient_2(f, args...)[i])
end

derivative_rule_gen (generic function with 1 method)

In [58]:
derivative_rule_gen(sin, 0, 1)

typeof(Base.sin)


sin (generic function with 14 methods)

In [60]:
include("autodiff_v2.jl")

derivative_rule (generic function with 3 methods)

In [66]:
derivative_rule(*, [3, 5], 1)

5

In [82]:
derivative_rule(*, [3, 5], 1)

5

In [81]:
derivative_rule_gen(mult, [3, 5], 1)

5

In [ ]:
MethodError: no method matching derivative_rule(::typeof(mult), ::Vector{Any}, ::Int64)

In [71]:
gradient_2(composite, 3, 1)

LoadError: MethodError: no method matching derivative_rule(::typeof(mult), ::Vector{Any}, ::Int64)
[0mClosest candidates are:
[0m  derivative_rule([91m::typeof(sin)[39m, ::Any, ::Any) at ~/Documents/MIT/18.S191/autodiff_v2.jl:63
[0m  derivative_rule([91m::typeof(+)[39m, ::Any, ::Any) at ~/Documents/MIT/18.S191/autodiff_v2.jl:67
[0m  derivative_rule([91m::typeof(*)[39m, ::Any, ::Any) at ~/Documents/MIT/18.S191/autodiff_v2.jl:71

In [30]:
@generated function foo(x)
           Core.println(x)
           return :(x * x)
       end

foo (generic function with 1 method)

In [91]:
include("autodiff_v3.jl")

derivative_rule (generic function with 4 methods)

In [92]:
function derivative_rule(::typeof(sin), args, i)
    return :(cos($(args[1])))
end

function derivative_rule(::typeof(+), args, i)
    return 1
end

function derivative_rule(::typeof(*), args, i)
    if i == 1
        return args[2]
    elseif i == 2
        return args[1]
    end
end

function derivative_rule(unknown_function, args, i)
    Core.println("fell through on", unknown_function, args)
    return gradient_3(unknown_function, args...)[i]
    # return derivative_rule_gen(unknown_function, args, i)
end

#@generated function derivative_rule_gen(f, args, i)
#end

derivative_rule (generic function with 4 methods)

In [94]:
gradient_3(composite, 3, 5)

Calling gradient onMain.composite(3, 5)
exExpr(:call, Main.mult, IRTools.Inner.Variable(id=2), IRTools.Inner.Variable(id=4))
fell through onMain.multArray{Any, (2,)}[IRTools.Inner.Variable(id=2), IRTools.Inner.Variable(id=4)]
Calling gradient onMain.mult(IRTools.Inner.Variable(id=2), IRTools.Inner.Variable(id=4))
exExpr(:call, Main.:(*), IRTools.Inner.Variable(id=2), IRTools.Inner.Variable(id=3))
factorIRTools.Inner.Variable(id=3)
exExpr(:call, Main.:(*), IRTools.Inner.Variable(id=2), IRTools.Inner.Variable(id=3))
factorIRTools.Inner.Variable(id=2)
finished transformation
1: (%1, %2, %3)
  %4 = %2 * %3
  %5 = 1
  %6 = 0
  %7 = derivative_rule(*, Any[%2, %3], i)
  %8 = %5 * %7
  %9 = %6 + %8
  %10 = 0
  %11 = derivative_rule(*, Any[%2, %3], i)
  %12 = %5 * %11
  %13 = %10 + %12
  %14 = Core.tuple(%4, %9, %13)
  return %14


LoadError: MethodError: no method matching *(::IRTools.Inner.Variable, ::IRTools.Inner.Variable)
[0mClosest candidates are:
[0m  *(::Any, ::Any, [91m::Any[39m, [91m::Any...[39m) at operators.jl:591
[0m  *([91m::ChainRulesCore.AbstractThunk[39m, ::Any) at ~/.julia/packages/ChainRulesCore/Z4Jry/src/tangent_arithmetic.jl:125
[0m  *([91m::ChainRulesCore.NoTangent[39m, ::Any) at ~/.julia/packages/ChainRulesCore/Z4Jry/src/tangent_arithmetic.jl:64
[0m  ...

In [84]:
derivative_rule(mult)

LoadError: MethodError: no method matching derivative_rule(::typeof(mult))
[0mClosest candidates are:
[0m  derivative_rule(::Any, [91m::Any[39m, [91m::Any[39m) at In[83]:17
[0m  derivative_rule([91m::typeof(sin)[39m, [91m::Any[39m, [91m::Any[39m) at In[83]:1
[0m  derivative_rule([91m::typeof(+)[39m, [91m::Any[39m, [91m::Any[39m) at In[83]:5
[0m  ...